# Web Scraping

By [Astrax Team](https://github.com/FTDS-assignment-bay/p2-final-project-rmt-040-final-sg02/blob/7ea7e3e9013f3bf3a45f3ad5e6d757fbb6e93547/scripts/scrapping.ipynb) | Data Resource: [FAQ Pajak](https://pajak.go.id/faq-page)

___

# Introduction

Program ini bertujuan untuk melakukan web scrapping pada halaman FAQ website [Pajak](https://pajak.go.id/id) dan mengekstraksi informasi yang relevan untuk membuat chatbot yang mampu menjawab pertanyaan-pertanyaan umum (FAQ).

# Import Libraries

In [2]:
import time
import re
import csv
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup as bs

In [3]:
# mengambil HTML dari halaman tertentu dan melakukan scrolling
def get_page_source(driver, url):
    driver.get(url)
    time.sleep(3) # tunggu 3 detik untuk load halaman
    for _ in range(3): # scroll halaman ke bawah 3 kali
        driver.execute_script("window.scrollBy(0, 250)")
        time.sleep(1)
    return driver.page_source

# mengekstrak daftar pertanyaan dan link dari halaman FAQ
def extract_questions_and_links(soup):
    question_list = []
    links = []
    for item in soup.find_all('div', class_='view-content'):
        # Ambil teks dari setiap pertanyaan
        raw_question = [q.get_text() for q in item.find_all('td', class_='views-field views-field-title')]
        # Bersihkan penomoran di depan pertanyaan
        cleaned_question = [re.sub(r'^\d{1,2}\.\s*', '', q).strip() for q in raw_question]
        question_list += cleaned_question

        # Link dari tiap pertanyaan
        links += [a['href'] for a in item.find_all('a', {"hreflang": "id"})]
    return question_list, links

# mengekstrak jawaban dari halaman detail berdasarkan link
def extract_answer(driver, link, current_question):
    full_link = f"https://pajak.go.id{link}" # melengkapi link yang dituju
    try:
        driver.get(full_link)
        time.sleep(2)
        soup_detail = bs(driver.page_source, "html.parser")
        content_div = soup_detail.find('div', class_='node__content')

        if not content_div:
            return "Tidak ada jawaban."

        # filter jawaban pada tiap link
        tables = content_div.find_all('table', class_="table table-striped table-bordered table-hover")
        tablist = content_div.find_all('h3', {"aria-expanded": "true"})
        paragraphs = content_div.find_all('p')

        # logic if-else
        if tables:
            return f'Informasi tentang "{current_question}" berbentuk table, buka link berikut untuk selengkapnya : {full_link}'
        elif tablist:
            return f'Informasi tentang "{current_question}" berbentuk tablist, buka link berikut untuk selengkapnya : {full_link}'
        elif len(paragraphs) > 3:
            return f'Informasi tentang "{current_question}" panjang untuk ditampilkan di sini. Silakan baca langsung di: {full_link}'

        # menghindari link yang memiliki kombinasi list urutan angka dan strip
        has_ol = content_div.find('ol') is not None
        has_ul = content_div.find('ul') is not None

        if has_ol and has_ul:
            return f'Informasi tentang "{current_question}" mengandung daftar kombinasi (bernomor dan bullet). Silakan baca langsung di: {full_link}'

        text_parts = []
        for elem in content_div.find_all(['p', 'ol', 'ul']):
            if elem.name == 'p':
                text_parts.append(elem.get_text(strip=True))
            elif elem.name == 'ol':
                for idx, li in enumerate(elem.find_all('li'), start=1):
                    text_parts.append(f"{idx}. {li.get_text(strip=True)}")
            elif elem.name == 'ul':
                for li in elem.find_all('li'):
                    text_parts.append(f"- {li.get_text(strip=True)}")
        
        # ambil link gambar
        images = content_div.find_all('img', {'data-entity-type': 'file'})
        image_links = []
        for img in images:
            src = img.get('src')
            if src:
                if src.startswith('/'):
                    src = f"https://pajak.go.id{src}"
                image_links.append(f"Gambar: {src}")

        # ambil link pdf
        a_tags = content_div.find_all('a', {"data-asw-orgfontsize": "14"})
        file_links = [f"Link PDF: https://pajak.go.id{a['href']}" if a['href'].startswith('/') else f"Link PDF: {a['href']}" for a in a_tags]

        # jika gambar lebih dari 1 tampilkan link
        if len(images) > 1:
            return f'Informasi tentang "{current_question}" dilengkapi beberapa gambar. Silahkan baca langsung melalui link berikut: {full_link}'

        # gabung semua jawaban
        final_answer = "\n".join(text_parts + image_links + file_links)

        # membatasi hasil jawaban yang diberikan
        max_length = 1000
        if len(final_answer) > max_length:
            return f'Jawaban untuk "{current_question}" terlalu panjang untuk ditampilkan. Silahkan baca langsung melalui link berikut: {full_link}'
        return "\n".join(text_parts + image_links + file_links)

    except Exception as e:
        return f"Gagal mengambil jawaban: {e}"

# fungsi untuk memulai scrapping
def scrape_faq(start_page=0, end_page=1):
    question = []
    answer = []

    driver = webdriver.Chrome()

    for i in range(start_page, end_page):
        url = f'https://pajak.go.id/id/faq-page?page={i}'
        html = get_page_source(driver, url)
        soup = bs(html, "html.parser")

        questions, links = extract_questions_and_links(soup) # ekstrak data
        question += questions

        for idx, link in enumerate(links):
            current_question = questions[idx]
            ans = extract_answer(driver, link, current_question)
            answer.append(ans)

    driver.quit()

    # simpan hasil ke csv
    df = pd.DataFrame({
        'Question': question,
        'Answer': answer
    })
    df.to_csv('hasil_scraping.csv', index=False, encoding='utf-8')  # menyimpan ke CSV

    print("Scraping selesai dan data disimpan ke 'hasil_scraping.csv'")

# jalankan scraping
scrape_faq(start_page=0, end_page=8)

Scraping selesai dan data disimpan ke 'hasil_scraping.csv'


In [5]:
pd.set_option('display.max_colwidth', None)  # Untuk menampilkan seluruh isi kolom
# pd.set_option('display.max_rows', None)      # Kalau mau menampilkan semua baris juga

In [6]:
df = pd.read_csv('hasil_scraping.csv')
df

,Question,Answer
0,Penggunaan NPWP 16 Digit pada Aplikasi SAKTI dan SPAN,Soal sering ditanya terkait Penggunaan NPWP 16 Digit pada Aplikasi SAKTI dan SPAN dapat diunduh melalui lampiran dibawah ini.\nLink PDF: https://pajak.go.id/system/files/2024-02/NPWP_16_Digit_SAKTI_SPAN.pdf.pdf
1,PMK-66 Tahun 2023 Terkait Natura,Link PDF: https://pajak.go.id/system/files/2023-12/FAQ%20Terkait%20PMK-66%20Tahun%202023.pdf
2,PMK-72 Tahun 2023 Terkait Amortisasi,Link PDF: https://pajak.go.id/system/files/2023-12/FAQ%20Terkait%20PMK-72%20Tahun%202023.pdf
3,Panduan Pemadanan NPWP Secara Langsung,"Layanan pemadanan Nomor Pokok Wajib Pajak (NPWP) dapat diberikan:\n1. secara elektronik melalui portal layanan, web service, akun pajak.go.id pada laman resmi DJP;\n2. secara langsung; dan\n3. melalui Penyedia Jasa Aplikasi Perpajakan.\nPanduan lebih lanjut Pemadanan NPWP Secara Langsung dapat dilihat pada dokumen terlampir di bawah ini.\nLink PDF: https://pajak.go.id/system/files/2023-09/Panduan%20Teknis%20Pemadanan%20NPWP%20Secara%20Langsung.pdf"
4,Penyampaian SPT Masa PPh Final dalam Rangka PPS,Link PDF: https://pajak.go.id/system/files/2023-08/FAQ%20SPT%20Masa%20PPh%20Final%20dalam%20Rangka%20PPS.pdf\nLink PDF: https://pajak.go.id/system/files/2023-08/Petunjuk%20Pengisian%20SPT%20Masa%20PPh%20Final%20dalam%20Rangka%20PPS.pdf
...,...,...
148,FAQ e-SPT Masa PPh Pasal 21-26 (PER-32/PJ/2009),Tidak ada jawaban.
149,FAQ e-SPT Masa PPh Pasal 21-26 2014 (PER-14/PJ/2013),"Jawaban untuk ""FAQ e-SPT Masa PPh Pasal 21-26 2014 (PER-14/PJ/2013)"" terlalu panjang untuk ditampilkan. Silahkan baca langsung melalui link berikut: https://pajak.go.id/id/faq-e-spt-masa-pph-pasal-21-26-2014-14pj2013"
150,FAQ e-SPT Masa PPh Pasal 15,"Jawaban untuk ""FAQ e-SPT Masa PPh Pasal 15"" terlalu panjang untuk ditampilkan. Silahkan baca langsung melalui link berikut: https://pajak.go.id/id/faq-e-spt-masa-pph-pasal-15"
151,FAQ e-SPT Masa Pph Pasal 4 Ayat (2),"Jawaban untuk ""FAQ e-SPT Masa Pph Pasal 4 Ayat (2)"" terlalu panjang untuk ditampilkan. Silahkan baca langsung melalui link berikut: https://pajak.go.id/id/faq-e-spt-masa-pph-pasal-4-ayat-2"
